<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

How often a certain product has been sold in the past days.

#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

    100% |████████████████████████████████| 87.4MB 290kB/s 
    100% |████████████████████████████████| 23.8MB 1.9MB/s 
    100% |████████████████████████████████| 2.7MB 14.7MB/s 
mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.2 which is incompatible.
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:
import turicreate as tc

# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

# 3)-Exploring dataset

In [7]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [8]:
transactions.shape

(2500, 2)

In [9]:
print(transactions['products'][1415])

[250, 236, 242, 229, 92, 2, 71, 172, 109, 247, 171, 209, 90, 139, 188, 191, 145, 214, 216, 237]


In [10]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 39.1+ KB


In [11]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


# 4)- Adding Features:

### 4.1)-Create a separate dataframe for recommending users

In [0]:
customers=transactions['id']

In [13]:
customers.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
import random
random.shuffle(customers) 

 To get random values of customer ids. This would help us in avoiding sample biaseness in our modeling.

In [0]:
cust_2_rec=customers[:1000]

In [16]:
cust_2_rec.head()

0    1127
1     526
2    2176
3    2081
4    2396
Name: id, dtype: int64

### 4.2)- break down each list of items in the products column into rows and count the number of products bought by a user

In [17]:
pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,526,34.0,1
1,526,45.0,1
2,526,82.0,1
3,526,108.0,1
4,526,113.0,1
5,526,119.0,1
6,526,148.0,1
7,526,157.0,1
8,526,165.0,1
9,1127,30.0,1


# 5- Creating features for user_id,product_id and purchase count

### 5a)-One for purchase count

In [18]:
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [19]:
data.shape

(24811, 3)

In [20]:
data.head()

,id,productId,purchase_count
0,0,3,1
1,0,23,1
2,0,27,1
3,0,84,1
4,0,100,1


In [21]:
data['productId'].value_counts(dropna=False).head(20)

36     127
248    127
175    122
154    122
85     119
21     118
48     118
126    118
17     117
7      117
174    117
235    116
152    116
122    116
134    116
150    116
23     116
3      116
70     115
2      115
Name: productId, dtype: int64

In [0]:
prod_freq=data['productId'].value_counts(dropna=False)

In [0]:
prod_freq.index.name
prod_freq.index.name='prod_ID'

In [0]:
def product_frequency(prod_ID):
    if prod_ID not in prod_freq.index:
        print('Product not found.')
        return prod_ID
    return prod_freq.loc[prod_ID]

In [26]:
prod_ID=int(input("enter a product ID to find frequency: "))

enter a product ID to find frequency: 2


In [27]:
print(product_frequency(prod_ID))

115


**This product frequency is not how each customer has purchased it. To add to our basic problem, we need to see how often a product has been purchased accross users. For this reason, I didn't end my assignment here.**

### 5-b)-Normalize item

we normalize purchase frequency of each item across users by first creating a user-item matrix 

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [29]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The NaN tells us that the item represented by the column was not purchased in that specific transaction.

In [30]:
df_matrix.shape

(2378, 250)

In [31]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
print(df_matrix_norm.shape)

(2378, 250)


### 5-c)- create a table for input to the modeling

In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [34]:
data_norm.head()

,id,productId,scaled_purchase_freq
15,15,1,1.0
20,20,1,0.0
80,86,1,0.0
98,104,1,0.0
123,132,1,0.0


In [35]:
data_norm.shape

(22530, 3)

### 5-d) a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()


we have normalized the items accoreding to their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item.

# 6)-Split train and test set

we have three dataframes with purchase counts(data), purchase dummy(data_dummy), and scaled purchase counts(data_norm).

In [37]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
#convert dataframe to SFrame
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

A tabular, column-mutable dataframe object that can scale to big data <br>
https://turi.com/products/create/docs/generated/graphlab.SFrame.html

In [39]:
train_data

id,productId,purchase_count
2211,141,1
2295,106,1
185,20,1
1483,139,1
405,65,1
359,192,1
997,217,1
2428,113,1
2417,209,1
1838,150,1


In [40]:
test_data

id,productId,purchase_count
648,166,1
553,58,1
1193,181,1
988,86,1
2360,198,1
2095,125,1
1051,16,1
1747,57,1
895,48,1
1655,244,1


### 6.2)- Define a split_data function 

In [0]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

### 6.3)-Apply for scaled/normalized purchase table

In [0]:
train_data_norm, test_data_norm = split_data(data_norm)

# 7)-Building Model

### 7.1)- Parameters to define field names for purchase count as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [44]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2337 users and 250 items.

Data prepared in: 0.029163s

19848 observations to process; with 250 unique items.

In [45]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 81659.3

+------+-----------+--------------------+------+
|  id  | productId |       score        | rank |
+------+-----------+--------------------+------+
| 1127 |    207    | 1.0975609756097562 |  1   |
| 1127 |     96    | 1.0886075949367089 |  2   |
| 1127 |    153    | 1.0833333333333333 |  3   |
| 1127 |     26    |        1.08        |  4   |
| 1127 |    112    | 1.0769230769230769 |  5   |
| 1127 |    101    | 1.0746268656716418 |  6   |
| 1127 |     85    | 1.0729166666666667 |  7   |
| 1127 |     35    | 1.0694444444444444 |  8   |
| 1127 |     71    | 1.064516129032258  |  9   |
| 1127 |     44    | 1.0638297872340425 |  10  |
| 526  |    207    | 1.0975609756097562 |  1   |
| 526  |     96    | 1.0886075949367089 |  2   |
| 526  |    153    | 1.0833333333333333 |  3   |
| 526  |     26    |        1.08        |  4   |
| 526  |    112    | 1.0769230769230769 |  5   |
| 526  |    101    | 1.0746268656716418 |  6   |
| 526  |     85    | 1.0729166666666667 |  7   |
| 526  |     35    |

### Note 

Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations.<br> These 30 records include 3 users and their recommended items, along with score and descending ranks.

## Checking most frequent items(as per assignment) 

In [46]:
 train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
207    1.097561
96     1.088608
153    1.083333
26     1.080000
112    1.076923
101    1.074627
85     1.072917
35     1.069444
71     1.064516
44     1.063830
152    1.063158
125    1.060976
150    1.060000
131    1.057471
122    1.054348
50     1.053191
129    1.051948
247    1.050633
156    1.050633
91     1.050633
Name: purchase_count, dtype: float64

**products 201,207,125,112,96 and 153 are the most popular (best-selling) across customers.**

### 7.2)- Applying 'scaled_purchase_freq' as target feature on model

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [48]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2338 users and 227 items.

Data prepared in: 0.026595s

18024 observations to process; with 227 unique items.

In [49]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 99950

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 1127 |     96    |        0.0875        |  1   |
| 1127 |    152    | 0.07865168539325842  |  2   |
| 1127 |     26    | 0.06666666666666667  |  3   |
| 1127 |    112    | 0.06329113924050633  |  4   |
| 1127 |    153    | 0.06172839506172839  |  5   |
| 1127 |    207    | 0.058823529411764705 |  6   |
| 1127 |    138    | 0.058823529411764705 |  7   |
| 1127 |     49    | 0.058823529411764705 |  8   |
| 1127 |    156    | 0.05813953488372093  |  9   |
| 1127 |    186    | 0.057971014492753624 |  10  |
| 526  |     96    |        0.0875        |  1   |
| 526  |    152    | 0.07865168539325842  |  2   |
| 526  |    125    | 0.06896551724137931  |  3   |
| 526  |     26    | 0.06666666666666667  |  4   |
| 526  |    112    | 0.06329113924050633  |  5   |
| 526  |    153    | 0.06172839506172839  |  6   |
| 526  |    207    | 0.05882352

Great!.... We have all three models worked out. We have purchase frequency of each item customer-vice. <br>
Which one to trust and which one to discard? 

# 8)- Evaluate models

### 8.1)- Validation for Popularity Model on Purchase Counts

In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [52]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1922 queries. users per second: 57208.2


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.011966701352757559 | 0.004960110995490803 |
|   2    | 0.01144640998959418  | 0.009399930627818243 |
|   3    | 0.010058966354491857 | 0.011692103133310221 |
|   4    | 0.010015608740894919 | 0.01640941149265813  |
|   5    | 0.011134235171696155 | 0.021798762862758732 |
|   6    | 0.011359694762400301 | 0.02783414267545378  |
|   7    | 0.011520737327188942 |  0.032234940455544   |
|   8    | 0.011381373569198749 | 0.03603430619559656  |
|   9    | 0.010926118626430803 | 0.038488347125183744 |
|   10   | 0.01087408949011446  | 0.04208826784269041  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.14827466442585885

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+

### 8.2)- Validation for Popularity Model on Scaled Purchase Counts

In [53]:
models_counts = [popularity_model_scaled]
model_names=['Popularity Model on Scaled Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_norm, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1840 queries. users per second: 59577


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.010869565217391308 | 0.0037642339544513453 |
|   2    | 0.013043478260869578 |  0.01062111801242237  |
|   3    | 0.011413043478260873 |  0.013610248447204952 |
|   4    | 0.011413043478260861 |  0.019941770186335382 |
|   5    | 0.011739130434782603 |  0.02465320910973083  |
|   6    | 0.011413043478260904 |  0.028312629399585922 |
|   7    | 0.01141304347826087  |  0.03337603519668737  |
|   8    | 0.011480978260869574 |  0.03898291925465854  |
|   9    | 0.011231884057971011 |  0.042895962732919214 |
|   10   |  0.0108695652173913  |  0.047415890269151065 |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.14582120000807897

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+-

 We tend to keep lowest rmse and it signifies least error in prediction. 



**So we will choose that as best model with rmse 0.147 i.e (in 8.1) for our final step.**

# 9) Submission

In [54]:
users_to_recommend = list(cust_2_rec)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_count')

Preparing data set.

Data has 24811 observations with 2378 users and 250 items.

Data prepared in: 0.031312s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 31.072ms                            | 100              | 250             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.03935s

In [55]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 34653.6

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 1127 |    168    | 0.025631868839263917 |  1   |
| 1127 |    248    | 0.025486195087432863 |  2   |
| 1127 |     23    | 0.024235220750172932 |  3   |
| 1127 |    126    | 0.022379732131958006 |  4   |
| 1127 |     32    | 0.02150735855102539  |  5   |
| 1127 |    214    | 0.021062183380126952 |  6   |
| 1127 |     63    | 0.020147351423899333 |  7   |
| 1127 |    139    | 0.019966328144073488 |  8   |
| 1127 |     74    | 0.01874724229176839  |  9   |
| 1127 |    110    | 0.018568030993143716 |  10  |
| 526  |     2     | 0.027535723315344915 |  1   |
| 526  |     17    | 0.027234587404463027 |  2   |
| 526  |    177    | 0.026982453134324815 |  3   |
| 526  |    214    | 0.025738835334777832 |  4   |
| 526  |     79    | 0.025663250022464328 |  5   |
| 526  |    112    | 0.025098118517133925 |  6   |
| 526  |     37    | 0.02449720

### 9.2)- Checking most frequent items in final model

In [56]:
data.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.080808
207    1.076923
153    1.071429
201    1.069767
26     1.063158
152    1.060345
125    1.058824
112    1.058824
85     1.058824
131    1.057143
35     1.056180
63     1.054545
44     1.054545
129    1.053763
101    1.053763
117    1.052632
150    1.051724
122    1.051724
156    1.051546
120    1.050000
Name: purchase_count, dtype: float64

**RESULT: <BR>
products 96,207 , 153,201 and 26(and so on) are the most frequent selling items across customers.**

# 10)- Bonus Part- Recommending products to customers

In [57]:
# Create a csv output file
df_rec = recom.to_dataframe()
df_rec.head()

,id,productId,score,rank
0,1127,168,0.025632,1
1,1127,248,0.025486,2
2,1127,23,0.024235,3
3,1127,126,0.022380,4
4,1127,32,0.021507,5


In [58]:
print(df_rec.shape)

(10000, 4)


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates().sort_values('id').set_index('id')

In [60]:
recomendation = final_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/1000 queries. users per second: 54347.8

In [0]:
df_rec = recomendation.to_dataframe()


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))

In [0]:
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('id').set_index('id')

In [64]:
df_output.head()

,recommendedProducts
id,
3,2|215|110|230|245|169|23|170|209|66
8,87|52|141|198|55|66|103|126|74|80
13,11|229|145|21|201|210|211|71|170|228
17,54|245|235|243|163|189|146|195|144|78
20,50|137|228|207|243|21|30|138|220|249


In [0]:
# define recommendation function
def customer_recomendation(id):
    if id not in df_output.index:
        print('Customer not found.')
        return id
    return df_output.loc[id]

In [66]:
customer_recomendation(48)

Customer not found.


48

In [67]:
customer_recomendation(2438)

recommendedProducts    235|241|72|125|14|128|216|121|120|62
Name: 2438, dtype: object

In [68]:
customer_recomendation(368)

Customer not found.


368